# 1. Exploratory Data Analysis

In [ ]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/thinkstats.py")

Downloaded thinkstats.py


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import HTML

import empiricaldist
from thinkstats import decorate

## 1.1 Evidence

**Anecdotal evidence** based on data that is unpublished and usually personal usually fails because:
- Small number of observations
- Selection bias
- Confirmation bias
- Inaccuracy

To address the limitations of anecdotes, we will use the tools of statistics:
- Data collection
- Descriptive statistics
- Exploratory data analysis
- Estimation
- Hypothesis testing

## 1.2. The National Survey of Family Growth

Since 1973 the U.S. Centers for Disease Control and Prevention (CDC) have conducted the National Survey of Family Growth (NSFG), which is intended to gather "information on family life, marriage and divorce, pregnancy, infertility, use of contraception, and men’s and women’s health. The survey results are used to plan health services and health education programs, and to do statistical studies of families, fertility, and health."

- https://www.cdc.gov/nchs/nsfg/index.htm

- The goal of a statistical study is to draw conclusions about a **population**.
- We collect data from a subset of the population called a **sample**.
- The people who participate in a survey are called **respondents**.
- The NSFG is a **cross-sectional** study, which means that it captures a snapshot of a population at a point in time.
- The NSFG has been conducted several times now; each deployment is called a **cycle**. We will use data from Cycle 6, which was conducted from January 2002 to March 2003.
- In general, cross-sectional studies are meant to be **representative**, which means that the sample is similar to the target population in all ways that are important for the purposes of the study.
- The NSFG is not representative; instead it is **stratified**, which means that it deliberately **oversamples** some groups.
- When working with this kind of data, it is important to be familiar with the **codebook**, which documents the design of the study, the survey questions, and the encoding of the responses.

The codebook and user’s guide for the NSFG data are available from https://www.cdc.gov/nchs/nsfg/nsfg_cycle6.htm

## 1.3. Reading the Data

In [ ]:
download("https://github.com/AllenDowney/ThinkStats/raw/v3/data/2002FemPreg.dct")
download("https://github.com/AllenDowney/ThinkStats/raw/v3/data/2002FemPreg.dat.gz")

Downloaded 2002FemPreg.dct
Downloaded 2002FemPreg.dat.gz


In [ ]:
import statadict

In [ ]:
dct_file = "2002FemPreg.dct"
dat_file = "2002FemPreg.dat.gz"

In [ ]:
from statadict import parse_stata_dict


def read_stata(dct_file, dat_file):
    stata_dict = parse_stata_dict(dct_file)
    resp = pd.read_fwf(
        dat_file,
        names=stata_dict.names,
        colspecs=stata_dict.colspecs,
        compression="gzip",
    )
    return resp

In [ ]:
preg = read_stata(dct_file, dat_file)

This `DataFrame` contains a row for each pregnancy reported by a respondent and a column for each **variable**. A variable can contain responses to a survey question or values that are calculated based on responses to one or more questions.

In [ ]:
preg.shape

(13593, 243)

This dataset contains 243 variables with information about 13,593 pregnancies. 

In [ ]:
preg.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,poverty_i,laborfor_i,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,0,3410.389399,3869.349602,6448.271112,2,9,1231
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,0,3410.389399,3869.349602,6448.271112,2,9,1231
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,0,0,0,0,7226.301740,8567.549110,12999.542264,2,12,1231
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,0,7226.301740,8567.549110,12999.542264,2,12,1231
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,0,7226.301740,8567.549110,12999.542264,2,12,1231


In [ ]:
preg.columns

Index(['caseid', 'pregordr', 'howpreg_n', 'howpreg_p', 'moscurrp', 'nowprgdk',
       'pregend1', 'pregend2', 'nbrnaliv', 'multbrth',
       ...
       'poverty_i', 'laborfor_i', 'religion_i', 'metro_i', 'basewgt',
       'adj_mod_basewgt', 'finalwgt', 'secu_p', 'sest', 'cmintvw'],
      dtype='object', length=243)

In [ ]:
pregordr = preg["pregordr"]
type(pregordr)

pandas.core.series.Series

In [ ]:
pregordr.head()

0    1
1    2
2    1
3    2
4    3
Name: pregordr, dtype: int64

The NSFG dataset contains 243 variables in total. Here are some of the ones we’ll use for the explorations in this book.

- `caseid` is the integer ID of the respondent.
- `pregordr` is a pregnancy serial number; for example, the code for a respondent’s first pregnancy is 1, for the second pregnancy is 2, and so on.
- `prglngth` is the integer duration of the pregnancy in weeks.
- `outcome` is an integer code for the outcome of the pregnancy. The code 1 indicates a live birth.
- `birthord` is a serial number for live births; the code for a respondent’s first child is 1, and so on. For outcomes other than live birth, this field is blank.
- `birthwgt_lb` and `birthwgt_oz` contain the pounds and ounces parts of the birth weight of the baby.
- `agepreg` is the mother’s age at the end of the pregnancy.
- `finalwgt` is the statistical weight associated with the respondent. It is a floating-point value that indicates the number of people in the U.S. population this respondent represents.

If you read the codebook carefully, you will see that many of the variables are **recodes**, which means that they are not part of the **raw data** collected by the survey – they are calculated using the raw data.

## 1.4. Validation

One way to validate data is to compute basic statistics and compare them with published results.

In [ ]:
preg["outcome"].value_counts().sort_index()

outcome
1    9148
2    1862
3     120
4    1921
5     190
6     352
Name: count, dtype: int64

In [ ]:
counts = preg["birthwgt_lb"].value_counts(dropna=False).sort_index()
counts

birthwgt_lb
0.0        8
1.0       40
2.0       53
3.0       98
4.0      229
5.0      697
6.0     2223
7.0     3049
8.0     1889
9.0      623
10.0     132
11.0      26
12.0      10
13.0       3
14.0       3
15.0       1
51.0       1
97.0       1
98.0       1
99.0      57
NaN     4449
Name: count, dtype: int64

In [ ]:
counts.loc[0:5]

birthwgt_lb
0.0      8
1.0     40
2.0     53
3.0     98
4.0    229
5.0    697
Name: count, dtype: int64

In [ ]:
counts.loc[0:5].sum()

np.int64(1125)

In [ ]:
preg["birthwgt_lb"] = preg["birthwgt_lb"].replace([51, 97, 98, 99], np.nan)

When you read data like this, you often have to check for errors and deal with special values. Operations like this are called **data cleaning**.

## 1.5. Transformation

`agepreg` contains the mother’s age at the end of the pregnancy. According to the codebook, it is an integer number of centiyears (hundredths of a year).

In [ ]:
preg["agepreg"].mean()

np.float64(2468.8151197039497)

To convert it to years, we can divide through by 100.

In [ ]:
preg["agepreg"] /= 100.0
preg["agepreg"].mean()

np.float64(24.6881511970395)

In [ ]:
preg["birthwgt_oz"].value_counts(dropna=False).sort_index()

birthwgt_oz
0.0     1037
1.0      408
2.0      603
3.0      533
4.0      525
5.0      535
6.0      709
7.0      501
8.0      756
9.0      505
10.0     475
11.0     557
12.0     555
13.0     487
14.0     475
15.0     378
97.0       1
98.0       1
99.0      46
NaN     4506
Name: count, dtype: int64

In [ ]:
preg["birthwgt_oz"] = preg["birthwgt_oz"].replace([97, 98, 99], np.nan)

1 lb = 16 oz ~ 0.45 kg

In [ ]:
preg["totalwgt_lb"] = preg["birthwgt_lb"] + preg["birthwgt_oz"] / 16.0
preg["totalwgt_lb"].mean()

np.float64(7.265628457623368)

## 1.6. Summary Statistics

A **statistic** is a number derived from a dataset, usually intended to quantify some aspect of the data. Examples include the count, mean, variance, and standard deviation.

In [ ]:
weights = preg["totalwgt_lb"]
n = weights.count()  # number of values that are not NaN
n

np.int64(9038)

In [ ]:
mean = weights.sum() / n
mean

np.float64(7.265628457623368)

In [ ]:
weights.mean()

np.float64(7.265628457623368)

In this dataset, the average birth weight is about 7.3 pounds.

Variance is a statistic that quantifies the spread of a set of values.

In [ ]:
squared_deviations = (weights - mean) ** 2

In [ ]:
var = squared_deviations.sum() / n
var

np.float64(1.983070989750022)

In [ ]:
weights.var()

1.9832904288326532

The result is *slightly different* because when the `var` method computes the mean of the squared deviations, it divides by `n-1` rather than `n`.

In [ ]:
weights.var(ddof=0)  # divide by n

1.9830709897500207

In this dataset, the variance of the birth weights is about 1.98, but that value is hard to interpret – for one thing, it is in units of pounds squared.

A better option is the **standard deviation**, which is the square root of variance.

In [ ]:
std = np.sqrt(var)
std

np.float64(1.40821553384062)

In [ ]:
weights.std(ddof=0)

1.4082155338406195

In this dataset, the standard deviation of birth weights is about 1.4 pounds. Informally, values that are one or two standard deviations from the mean are common – values farther from the mean are rare.

## 1.7. Interpretation

In [ ]:
subset = preg.query("caseid == 10229")
subset.shape

(7, 244)

This respondent reported seven pregnancies – here are their outcomes, which are recorded in chronological order.

In [ ]:
subset["outcome"].values

array([4, 4, 4, 4, 4, 4, 1])

The outcome code `1` indicates a live birth. Code `4` indicates a miscarriage – that is, a pregnancy loss, usually with no known medical cause.